<a href="https://colab.research.google.com/github/marcocintra/Atmosphere/blob/master/CASE_STUDY_MAGGIA_MAPS_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ZIP FILES ADJUSTS

In [ ]:
!curl 'https://my.microsoftpersonalcontent.com/personal/a149071e7d7aaa61/_layouts/15/download.aspx?UniqueId=e43fcca5-9432-40f2-bc9e-3d8a5d991204&Translate=false&tempauth=v1e.eyJzaXRlaWQiOiI3NzM0Y2NkNy1mMWE2LTQ0ZmItOWZmYy03MjIyYzI4NGE0ODEiLCJhcHBpZCI6IjAwMDAwMDAwLTAwMDAtMDAwMC0wMDAwLTAwMDA0ODE3MTBhNCIsImF1ZCI6IjAwMDAwMDAzLTAwMDAtMGZmMS1jZTAwLTAwMDAwMDAwMDAwMC9teS5taWNyb3NvZnRwZXJzb25hbGNvbnRlbnQuY29tQDkxODgwNDBkLTZjNjctNGM1Yi1iMTEyLTM2YTMwNGI2NmRhZCIsImV4cCI6IjE3NDU1MjI3MzcifQ.hgLqBxgVoMvx7GgJZUjXPQ3coC09gMLvqvOFrtu-ekbOlhPI2xxtq8RkTeoAxo73nH8k8rau8WIcHJ3L4geWWuKPKWXnxIA7gbZnMAuGuUTfz1eZhj_sNPzZT8dDZhb1VjK_ZtVU90VQmp4fHiL9aa0O5jsKvojsAm4HnNUOC8Vppwd6jd-2QEKuTfQfZFFKNSjdCQo4gl5U2oQaDniAW8Dw2G1Nau2haybmSJjs5eb6BUH6p3mkilQ6GiKfvwfPRm-UuIZlnOkqAeyS08rmrNyj0F15juyGQ2CwUafM81eSuJbNT8lk0Gad5s80ZWtcIZdqoByNWCOM-KVw9faBosJeC0wAepwtGVMuFEH71JmbUV87lkGVb-3jQ9OkudiYr8Uj89Mo9_ct2xoFybB2uio9Cn0B_vD0PQvwmg62xr4.kkvBStYCDo1m0DX6TMC9xMIOCCVvMahHA7Ref1ibGGE&ApiVersion=2.0&AVOverride=1' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
  -H 'accept-language: pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'priority: u=0, i' \
  -H 'referer: https://onedrive.live.com/' \
  -H 'sec-ch-ua: "Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Linux"' \
  -H 'sec-fetch-dest: iframe' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-site: cross-site' \
  -H 'sec-fetch-storage-access: none' \
  -H 'sec-fetch-user: ?1' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36' > 'MAGGIA-SEPT-05-2024-TO-END-2024.zip'

In [ ]:
%pwd

In [ ]:
!ls

In [ ]:
!unzip -q 'MAGGIA-SEPT-05-2024-TO-END-2024.zip'

In [ ]:
%pwd

In [ ]:
!ls

# DATA FILES ADJUSTS

In [ ]:
%cd MAGGIA-SEPT-05-2024-TO-END-2024

In [ ]:
import os
import glob
from collections import defaultdict

base_path = './'

file_pattern = os.path.join(base_path, '**', 'magn2024*.nc')
file_list = sorted(glob.glob(file_pattern, recursive=True))

file_count_by_day = defaultdict(int)

for file_name in file_list:
    day_dir = os.path.basename(os.path.dirname(file_name))
    file_count_by_day[day_dir] += 1

print("\nDias com menos de 96 arquivos:")
for day, count in sorted(file_count_by_day.items()):
    if count < 96:
        print(f"Dia {day}: {count} arquivos")

In [ ]:
!pip install netCDF4

In [ ]:
!pip install xarray

In [ ]:
import numpy as np
import xarray as xr
import os
import re
import glob

file_list = glob.glob("./magn2024*.nc")

file_list.sort()

tec_by_doy = {}

pattern = re.compile(r'magn2024(\d{3})')

for file_name in file_list:
    try:
        match = pattern.search(file_name)
        if match:
            doy = match.group(1)

            print(f"Processando o arquivo: {file_name}, DOY: {doy}")

            ds_disk = xr.open_dataset(file_name)
            tec = ds_disk['z'][:, :].values

            if doy not in tec_by_doy:
                tec_by_doy[doy] = []

            tec_by_doy[doy].append(tec)

            ds_disk.close()

    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

print("\nSalvando arquivos NPY por dia...")
for doy, tec_list in tec_by_doy.items():
    tec_array = np.array(tec_list)

    output_file = f"magn2024{doy}.npy"
    np.save(output_file, tec_array)

    print(f"Arquivo {output_file} salvo com sucesso! Shape: {tec_array.shape}")

print("\nProcessamento concluído!")

In [ ]:
!ls -lR ./*.npy | wc -l

In [ ]:
import numpy as np
import glob
import re

npy_files = glob.glob("magn2024*.npy")

def extract_doy(filename):
    match = re.search(r'magn2024(\d{3})\.npy', filename)
    if match:
        return int(match.group(1))
    return 0

npy_files.sort(key=extract_doy)

print(f"Encontrados {len(npy_files)} arquivos NPY para processar.")

all_data = []

for i, file in enumerate(npy_files):
    doy = extract_doy(file)
    print(f"Processando {file} (DOY {doy}) - {i+1}/{len(npy_files)}")

    data = np.load(file)

    all_data.append(data)

    print(f"  Shape: {data.shape}")

combined_data = np.concatenate(all_data, axis=0)

print(f"\nDados combinados. Shape final: {combined_data.shape}")

output_file = "MAGGIA-2024.npy"
np.save(output_file, combined_data)

print(f"\nArquivo {output_file} salvo com sucesso!")
print(f"Tamanho do arquivo: {combined_data.nbytes / (1024**2):.2f} MB")

In [ ]:
%cd './'

In [ ]:
!ls

In [ ]:
%pwd

In [ ]:
!ls *.npy

In [ ]:
maggia_2024 = np.load('MAGGIA-2024.npy')
print(np.shape(maggia_2024))
print(type(maggia_2024))
print(type(maggia_2024[0]))
print(type(maggia_2024[0][0]))
print(type(maggia_2024[0][0][0]))

In [ ]:
np.shape(maggia_2024)

In [ ]:
has_nan = np.any(np.isnan(maggia_2024))
print(has_nan)

count_nan = np.sum(np.isnan(maggia_2024))
print(count_nan)

In [ ]:
maggia_2024.shape

In [ ]:
np.set_printoptions(threshold=np.inf)

In [ ]:
print(maggia_2024[0][0])

In [ ]:
print(maggia_2024[0][-1])

In [ ]:
print(type(maggia_2024))
print(type(maggia_2024[0]))
print(type(maggia_2024[0][0]))
print(type(maggia_2024[0][0][0]))

In [ ]:
for i in range(len(maggia_2024)):
    print(np.shape(maggia_2024[i]))

In [ ]:
np.shape(maggia_2024)

In [ ]:
has_nan = np.any(np.isnan(maggia_2024))
print(has_nan)

count_nan = np.sum(np.isnan(maggia_2024))
print(count_nan)

# DATES ADJUSTS

In [ ]:
import numpy as np
from datetime import datetime, timedelta

def doy_to_date(year, doy):
    """Convert year and day of year to datetime."""
    return datetime(year, 1, 1) + timedelta(days=doy - 1)

doys = {

    2024: {
        9: [249, 250, 251, 252, 253, 254, 255, 256, 257, 258,
            261, 262, 263, 264, 265, 266, 267, 268, 269, 270,
            271, 272, 274],
        10: [275, 276, 277, 278, 279, 281, 282, 283, 285, 286,
             287, 288, 289, 290, 291, 292, 293, 295, 299, 300,
             301, 302, 303, 304, 305],
        11: [306, 307, 308, 309, 310, 311, 312, 313, 314, 315,
             316, 317, 318, 320, 321, 322, 323, 324, 325, 326,
             327, 328, 329, 330, 331, 332, 333, 334, 335],
        12: [336, 337, 338, 339, 340, 341, 342, 343, 344, 345,
             346, 347, 348, 349, 350, 351, 352, 353, 354, 355,
             356, 357, 358, 359, 360, 361, 362, 363, 364, 365,
             366]
    }
}

doy249_times = [
    "0000", "0010", "0015", "0020", "0030", "0040", "0045", "0050",
    "0100", "0110", "0115", "0120", "0130", "0140", "0145", "0150",
    "0200", "0210", "0215", "0220", "0230", "0240", "0245", "0250",
    "0300", "0310", "0315", "0320", "0330", "0340", "0345", "0350",
    "0400", "0410", "0415", "0420", "0430", "0440", "0445", "0450",
    "0500", "0510", "0515", "0520", "0530", "0540", "0545", "0550",
    "0600", "0610", "0615", "0620", "0630", "0640", "0645", "0650",
    "0700", "0710", "0715", "0720", "0730", "0740", "0745", "0750",
    "0800", "0810", "0815", "0820", "0830", "0840", "0845", "0850",
    "0900", "0910", "0915", "0920", "0930", "0940", "0945", "0950",
    "1000", "1010", "1015", "1020", "1030", "1040", "1045", "1050",
    "1100", "1110", "1115", "1120", "1130", "1140", "1145", "1150",
    "1200", "1210", "1215", "1220", "1230", "1240", "1245", "1250",
    "1300", "1310", "1315", "1320", "1330", "1340", "1345", "1350",
    "1400", "1410", "1415", "1420", "1430", "1440", "1445", "1450",
    "1500", "1510", "1515", "1520", "1530", "1540", "1545", "1550",
    "1600", "1610", "1615", "1620", "1630", "1640", "1645", "1650",
    "1700", "1710", "1715", "1720", "1730", "1740", "1745", "1750",
    "1800", "1810", "1815", "1820", "1830", "1840", "1845", "1850",
    "1900", "1910", "1915", "1920", "1930", "1940", "1945", "1950",
    "2000", "2010", "2015", "2020", "2030", "2040", "2045", "2050",
    "2100", "2110", "2115", "2120", "2130", "2140", "2145", "2150",
    "2200", "2210", "2215", "2220", "2230", "2240", "2245", "2250",
    "2300", "2310", "2315", "2320", "2330", "2340", "2345", "2350"
]

doy250_times = [
    "0000", "0010", "0015", "0020", "0030", "0040", "0045", "0050",
    "0100", "0110", "0115", "0120", "0130", "0140", "0145", "0150",
    "0200", "0210", "0215", "0220", "0230", "0240", "0245", "0250",
    "0300", "0310", "0315", "0320", "0330", "0340", "0345", "0350",
    "0400", "0410", "0415", "0420", "0430", "0440", "0445", "0450",
    "0500", "0510", "0515", "0520", "0530", "0540", "0545", "0550",
    "0600", "0610", "0615", "0620", "0630", "0640", "0645", "0650",
    "0700", "0710", "0715", "0720", "0730", "0740", "0745", "0750",
    "0800", "0810", "0815", "0820", "0830", "0840", "0845", "0850",
    "0900", "0910", "0915", "0920", "0930", "0940", "0945", "0950",
    "1000", "1010", "1015", "1020", "1030", "1040", "1045", "1050",
    "1100", "1110", "1115", "1120", "1130", "1140", "1145", "1150",
    "1200", "1210", "1220", "1230", "1240", "1250", "1300", "1310",
    "1320", "1330", "1340", "1350", "1400", "1410", "1420", "1430",
    "1440", "1450", "1500", "1510", "1520", "1530", "1540", "1550",
    "1600", "1610", "1620", "1630", "1640", "1650", "1700", "1710",
    "1720", "1730", "1740", "1750", "1800", "1810", "1820", "1830",
    "1840", "1850", "1900", "1910", "1920", "1930", "1940", "1950",
    "2000", "2010", "2020", "2030", "2040", "2050", "2100", "2110",
    "2120", "2130", "2140", "2150", "2200", "2210", "2220", "2230",
    "2240", "2250", "2300", "2310", "2320", "2330", "2340", "2350"
]

datetime_list = []

for year in doys.keys():
    for month in doys[year].keys():
        for doy in doys[year][month]:
            base_date = doy_to_date(year, doy)

            if year == 2024 and doy == 249:
                for time_str in doy249_times:
                    hours = int(time_str[:2])
                    minutes = int(time_str[2:])
                    dt = base_date + timedelta(hours=hours, minutes=minutes)
                    datetime_list.append(dt)

            elif year == 2024 and doy == 250:
                for time_str in doy250_times:
                    hours = int(time_str[:2])
                    minutes = int(time_str[2:])
                    dt = base_date + timedelta(hours=hours, minutes=minutes)
                    datetime_list.append(dt)

            elif year == 2024 and doy >= 251:
                for minute in range(0, 24 * 60, 10):
                    dt = base_date + timedelta(minutes=minute)
                    datetime_list.append(dt)

            else:
                for minute in range(0, 24 * 60, 15):
                    dt = base_date + timedelta(minutes=minute)
                    datetime_list.append(dt)

maggia_datetime_2024 = np.array(datetime_list, dtype='datetime64[s]')

print(f"Total number of datetime points: {len(maggia_datetime_2024)}")
print(f"First datetime: {maggia_datetime_2024[0]}")
print(f"Last datetime: {maggia_datetime_2024[-1]}")

np.save('maggia_datetime_2024.npy', maggia_datetime_2024)

print("\nSample of first 10 datetime entries:")
for dt in maggia_datetime_2024[:10]:
    print(dt)

print("\nSample of last 10 datetime entries:")
for dt in maggia_datetime_2024[-10:]:
    print(dt)

In [ ]:
maggia_dates = np.load('maggia_datetime_2024.npy')

In [ ]:
maggia_dates.shape

In [ ]:
maggia_dates

In [ ]:
maggia_2024.shape

In [ ]:
print(len(maggia_2024.tolist()))
print(len(maggia_2024[0].tolist()))
print(len(maggia_2024[0][0].tolist()))

In [ ]:
maggia_2024.shape

In [ ]:
len(maggia_dates)

In [ ]:
maggia_dates[0:12]

In [ ]:
len(maggia_dates[0:12])

In [ ]:
len(maggia_2024[0:12])

In [ ]:
maggia_2024_shaped = np.reshape(maggia_2024, (-1, 241, 221))

In [ ]:
np.shape(maggia_2024_shaped)

In [ ]:
np.shape(maggia_2024_shaped.tolist())

In [ ]:
maggia_dates.shape

## CREATE DATAFRAME

In [ ]:
import pandas as pd

data = {
    'DATETIME': maggia_dates,
    'TECMAP': maggia_2024_shaped.tolist()
}

df_mapas_maggia_2024 = pd.DataFrame(data)

df_mapas_maggia_2024

In [ ]:
np.array(df_mapas_maggia_2024.iloc[0]['TECMAP'])

In [ ]:
np.array(df_mapas_maggia_2024.iloc[0]['TECMAP']).shape

In [ ]:
df_mapas_maggia_2024.to_pickle("./df_mapas_maggia_sept_05_2024_beyond.pkl")

In [ ]:
mapas_maggia_2024 = np.array(df_mapas_maggia_2024.iloc[:]['TECMAP'])

In [ ]:
np.shape(df_mapas_maggia_2024)

In [ ]:
np_mapas_maggia_2024 = []
for i in range(len(mapas_maggia_2024)):
    np_mapas_maggia_2024.append(mapas_maggia_2024[i])
np_mapas_maggia_2024 = np.array(np_mapas_maggia_2024)

In [ ]:
np.shape(np_mapas_maggia_2024)

In [ ]:
type(np_mapas_maggia_2024)

In [ ]:
np.save('mapas_maggia_sept_05_2024_beyond.npy', np_mapas_maggia_2024)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls -lh mapas*

In [ ]:
!ls -lh df_mapas*

In [ ]:
%cp /content/MAGGIA-SEPT-05-2024-TO-END-2024/df_mapas_maggia_sept_05_2024_beyond.pkl /content/drive/MyDrive/df_mapas_maggia_sept_05_2024_beyond.pkl

In [ ]:
%cp /content/MAGGIA-SEPT-05-2024-TO-END-2024/mapas_maggia_sept_05_2024_beyond.npy /content/drive/MyDrive/mapas_maggia_sept_05_2024_beyond.npy